<a href="https://colab.research.google.com/github/akashvb1/CODSOFT/blob/main/CP_Module_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from google.colab import drive
import cv2
from IPython.display import display
from ipywidgets import widgets

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_path = '/content/drive/MyDrive/cows'
img_size = 224
batch_size = 32

In [ ]:
def load_data(dataset_path, img_size):
    X, y = [], []
    classes = os.listdir(dataset_path)
    class_names = sorted(classes)
    class_mapping = {breed: idx for idx, breed in enumerate(class_names)}
    for breed in classes:
        breed_path = os.path.join(dataset_path, breed)
        for img_file in os.listdir(breed_path):
            img_path = os.path.join(breed_path, img_file)
            try:
                img = load_img(img_path, target_size=(img_size, img_size))
                img_array = img_to_array(img) / 255.0
                X.append(img_array)
                y.append(class_mapping[breed])
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
    return np.array(X), np.array(y), class_names
print("Loading dataset...")
X, y, class_names = load_data(dataset_path, img_size)
print(f"Loaded {len(X)} images across {len(class_names)} classes: {class_names}")

Loading dataset...
Loaded 28000 images across 14 classes: ['Bargur', 'Deoni', 'Gir', 'Hallikar', 'Holstein Friesian', 'Jersey', 'Kangayam', 'Malanad', 'Ongole', 'Punganur', 'Red Sindhi', 'Sahiwal', 'Tharparkar', 'Vechur']


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = to_categorical(y_train, num_classes=len(class_names))
y_test = to_categorical(y_test, num_classes=len(class_names))

In [ ]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 610.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 92.7 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 179, in resolve
    self.factory.preparer.prepare_linked_requirements_more(reqs)
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/prepare.py", lin

In [ ]:
!pip install --upgrade tensorflow # Make sure TensorFlow is up to date
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' # Allow GPU memory growth to resolve allocation issues

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=4, batch_size=batch_size)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 186624)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      23,888,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 14)                  │           1,806 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,909,198 (91.21 MB)

 Trainable params: 23,909,198 (91.21 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/4
700/700 ━━━━━━━━━━━━━━━━━━━━ 2034s 3s/step - accuracy: 0.1708 - loss: 2.6696 - val_accuracy: 0.3121 - val_loss: 1.9717
Epoch 2/4
397/700 ━━━━━━━━━━━━━━━━━━━━ 13:46 3s/step - accuracy: 0.3378 - loss: 1.9473

In [ ]:
model_save_path = "/content/drive/MyDrive/cow_breed_model.h5"
model.save(model_save_path)

In [ ]:
from IPython.display import display
def predict_image(image_path, model, class_names, img_size):
    img = load_img(image_path, target_size=(img_size, img_size))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array)
    max_prob = np.max(predictions)
    predicted_class = class_names[np.argmax(predictions)]
    confidence_threshold = 0.8
    if max_prob < confidence_threshold:
        predicted_class = None

    probabilities = {class_names[i]: round(prob * 100, 2) for i, prob in enumerate(predictions[0])}
    return predicted_class, probabilities, img


In [ ]:
import ipywidgets as widgets
from IPython.display import display
def file_upload_prediction():
    print("Upload a cow image for prediction:")
    uploaded = widgets.FileUpload(accept='image/*', multiple=False)
    display(uploaded)

    def on_upload(change):
        for name, file_info in uploaded.value.items():
            img_path = f"/content/{name}"
            with open(img_path, 'wb') as f:
                f.write(file_info['content'])
            pred_class, probs, img = predict_image(img_path, model, class_names, img_size)
            print(f"Predicted Breed: {pred_class}")
            print("Probabilities:", probs)
            plt.imshow(img)
            plt.title(f"Prediction: {pred_class}")
            plt.axis('off')
            plt.show()
    uploaded.observe(on_upload, 'value')


In [ ]:
from google.colab.output import eval_js
from base64 import b64decode
from PIL import Image
import io
import matplotlib.pyplot as plt
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np


class CameraPrediction:
    def __init__(self, model, class_names, img_size):
        self.model = model
        self.class_names = class_names
        self.img_size = img_size

    def take_photo(self, quality=0.8):
        js = """
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture Image';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            div.appendChild(video);

            document.body.appendChild(div);

            const stream = await navigator.mediaDevices.getUserMedia({ video: true });
            video.srcObject = stream;
            await video.play();

            video.style.width = '300px';
            video.style.height = '300px';

            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);

            stream.getTracks().forEach(track => track.stop());
            video.remove();
            capture.remove();
            div.remove();

            const dataURL = canvas.toDataURL('image/jpeg', quality);
            return dataURL;
        }
        takePhoto(%f);
        """ % quality
        return eval_js(js)

    def predict_image_camera(self):
        print("Capturing image from webcam...")
        try:
            image_data = self.take_photo()
            image_bytes = b64decode(image_data.split(',')[1])
            img = Image.open(io.BytesIO(image_bytes))
            img_path = "/content/captured_image.jpg"
            img.save(img_path)
            pred_class, probs, _ = predict_image(img_path, self.model, self.class_names, self.img_size)
            print(f"Predicted Breed: {pred_class}")
            print("Probabilities:", probs)
            plt.imshow(img)
            plt.title(f"Prediction: {pred_class}")
            plt.axis('off')
            plt.show()
        except Exception as e:
            print(f"Error capturing image: {e}")
def camera_prediction():
    model = tf.keras.models.load_model(model_save_path)
    class_names = ['Bargur', 'Deoni', 'Gir', 'Hallikar', 'Holstein Friesian', 'Jersey', 'Kangayam', 'Malanad', 'Ongole', 'Punganur', 'Red Sindhi', 'Sahiwal', 'Tharparkar', 'Vechur']
    img_size = 224
    camera_pred = CameraPrediction(model, class_names, img_size)
    camera_pred.predict_image_camera()

In [ ]:
print("Choose input method:\n1. File Upload\n2. Camera Input")
choice = input("Enter choice (1/2): ")

if choice == '1':
    print("File upload option selected. Implement file upload logic.")
    file_upload_prediction()
elif choice == '2':
  camera_prediction()
else:
    print("Invalid choice!")